In [2]:
import psycopg2 as psycopg
import pandas as pd
from dotenv import load_dotenv
import os

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
load_dotenv(dotenv_path=".env")
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST", ""),
    "port": os.getenv("DB_DESTINATION_PORT", ""),
    "dbname": os.getenv("DB_DESTINATION_NAME", ""),
    "user": os.getenv("DB_DESTINATION_USER", ""),
    "password": os.getenv("DB_DESTINATION_PASSWORD", ""),
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определяем название таблицы, в которой хранятся наши данные
TABLE_NAME = "users_churn"


# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций с базой данных
# причём закрыто оно будет даже в случае ошибки при работе с базой данных
# это нужно, чтобы не допустить так называемую "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаём объект курсора для выполнения запросов к базе данных 
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
				
				# извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

				# получаем список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаём объект DataFrame из полученных данных и имён столбцов 
# это позволяет удобно работать с данными в Python с использованием библиотеки Pandas
df = pd.DataFrame(data, columns=columns)

print(f"Размер нашей таблицы: {df.shape[0]} строк; {df.shape[1]} столбцов")

Размер нашей таблицы: 7043 строк; 22 столбцов


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 7043 non-null   int64         
 1   customer_id        7043 non-null   object        
 2   begin_date         7043 non-null   datetime64[ns]
 3   end_date           1869 non-null   datetime64[ns]
 4   type               7043 non-null   object        
 5   paperless_billing  7043 non-null   object        
 6   payment_method     7043 non-null   object        
 7   monthly_charges    7043 non-null   float64       
 8   total_charges      7032 non-null   float64       
 9   internet_service   5517 non-null   object        
 10  online_security    5517 non-null   object        
 11  online_backup      5517 non-null   object        
 12  device_protection  5517 non-null   object        
 13  tech_support       5517 non-null   object        
 14  streamin

In [4]:
columns = [
    "customer_id", "begin_date", "end_date", "type", "paperless_billing",
    "payment_method", "monthly_charges", "total_charges", "internet_service",
    "online_security", "online_backup", "device_protection", "tech_support",
    "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner",
    "dependents", "multiple_lines", "target"]

with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(",".join(columns))

In [5]:
columns = [
    "customer_id", "begin_date", "end_date", "type", "paperless_billing",
    "payment_method", "monthly_charges", "total_charges", "internet_service",
    "online_security", "online_backup", "device_protection", "tech_support",
    "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner",
    "dependents", "multiple_lines", "target"]

stats = {}

for col in columns:
	# посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}
    stats


stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max() # посчитайте максимальное значение в колонке
stats["monthly_charges_mean"] = df["monthly_charges"].mean() # посчитайте среднее значение в колонке
stats["monthly_charges_median"] = df["monthly_charges"].median() # посчитайте медианное значение в колонке
stats["total_charges_min"] = df["total_charges"].min() # посчитайте минимальное значение в колонке
stats["total_charges_max"] = df["total_charges"].max() # посчитайте максимальное значение в колонке
stats["total_charges_mean"] = df["total_charges"].mean() # посчитайте среднее значение в колонке
stats["total_charges_median"] = df["total_charges"].median() # посчитайте медианное значение в колонке
stats["unique_customers_number"] = df["customer_id"].nunique() # посчитайте кол-во уникальных id
stats["end_date_nan"] = df["end_date"].isna().sum() # посчитайте кол-во пустых строк в колонке

In [6]:
df.to_csv("users_churn.csv", index=False) 

In [7]:
import mlflow
import os
EXPERIMENT_NAME='test_connection_experiment_vadim_shakula'
RUN_NAME='baseline_logreg_v1'

# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
mlflow.set_experiment(EXPERIMENT_NAME)
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id
    
    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats)
    
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact('columns.txt', "dataframe")
    mlflow.log_artifact('users_churn.csv', "dataframe")


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id) 


# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert (run.info.status =='FINISHED')

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove('columns.txt') 
os.remove('users_churn.csv')

/home/mle-user/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/mle-user/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [8]:
print(experiment_id)
print(EXPERIMENT_NAME)
print(run_id)

548588041841417201
test_connection_experiment_vadim_shakula
3ff405c5e1e147a3893dc517c866f1ff


In [9]:
def create_connection():

    load_dotenv()
    host = os.environ.get('DB_DESTINATION_HOST')
    port = os.environ.get('DB_DESTINATION_PORT')
    db = os.environ.get('DB_DESTINATION_NAME')
    username = os.environ.get('DB_DESTINATION_USER')
    password = os.environ.get('DB_DESTINATION_PASSWORD')
    
    print(f'postgresql://{username}:{password}@{host}:{port}/{db}')
    conn = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db}', connect_args={'sslmode':'require'})
    return conn

In [10]:
sql = f"""select count(*) 
from experiments 
join runs on experiments.experiment_id = runs.experiment_id 
where experiments.experiment_id = 548588041841417201
  and experiments.name = 'test_connection_experiment_vadim_shakula' 
  and runs.lifecycle_stage = 'active'"""

In [11]:
sql = f"""select experiments.name
from experiments 
join runs on experiments.experiment_id = runs.experiment_id 
where experiments.name = 'test_connection_experiment_vadim_shakula'
    and runs.lifecycle_stage = 'active'"""

In [12]:
dt = pd.read_sql_query(sql, conn)

/tmp/ipykernel_175568/445849524.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dt = pd.read_sql_query(sql, conn)


In [13]:
print(dt)

                                        name
0   test_connection_experiment_vadim_shakula
1   test_connection_experiment_vadim_shakula
2   test_connection_experiment_vadim_shakula
3   test_connection_experiment_vadim_shakula
4   test_connection_experiment_vadim_shakula
5   test_connection_experiment_vadim_shakula
6   test_connection_experiment_vadim_shakula
7   test_connection_experiment_vadim_shakula
8   test_connection_experiment_vadim_shakula
9   test_connection_experiment_vadim_shakula
10  test_connection_experiment_vadim_shakula


In [15]:
sqll = f"""select id,customer_id,begin_date,end_date,
type,
paperless_billing,
payment_method,
monthly_charges,
total_charges,
internet_service,
online_security,
online_backup,
device_protection,
tech_support,
streaming_tv,
streaming_movies,
gender,
senior_citizen,
partner, 
dependents,
multiple_lines,
target
from users_churn"""
date = pd.read_sql_query(sqll, conn)


/tmp/ipykernel_175568/1913469447.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  date = pd.read_sql_query(sqll, conn)


In [16]:
print(date)

         id customer_id begin_date   end_date            type   
0         4  7795-CFOCW 2016-05-01        NaT        One year  \
1         5  9237-HQITU 2019-09-01 2019-11-01  Month-to-month   
2         6  9305-CDSKC 2019-03-01 2019-11-01  Month-to-month   
3         7  1452-KIOVK 2018-04-01        NaT  Month-to-month   
4         8  6713-OKOMC 2019-04-01        NaT  Month-to-month   
...     ...         ...        ...        ...             ...   
7038  13028  0550-DCXLH 2019-01-01        NaT  Month-to-month   
7039  13029  9281-CEDRU 2014-06-01        NaT        Two year   
7040  13030  2235-DWLJU 2019-08-01        NaT  Month-to-month   
7041  13031  0871-OPBXW 2019-12-01        NaT  Month-to-month   
7042  13032  3605-JISKB 2015-07-01        NaT        One year   

     paperless_billing             payment_method  monthly_charges   
0                   No  Bank transfer (automatic)            42.30  \
1                  Yes           Electronic check            70.70   
2        

In [17]:
date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 7043 non-null   int64         
 1   customer_id        7043 non-null   object        
 2   begin_date         7043 non-null   datetime64[ns]
 3   end_date           1869 non-null   datetime64[ns]
 4   type               7043 non-null   object        
 5   paperless_billing  7043 non-null   object        
 6   payment_method     7043 non-null   object        
 7   monthly_charges    7043 non-null   float64       
 8   total_charges      7032 non-null   float64       
 9   internet_service   5517 non-null   object        
 10  online_security    5517 non-null   object        
 11  online_backup      5517 non-null   object        
 12  device_protection  5517 non-null   object        
 13  tech_support       5517 non-null   object        
 14  streamin